In [51]:
!pip install arviz


You should consider upgrading via the '/Users/alexkotlar/miniconda3/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([3., 2.]), afMean = tensor(1e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

covShared tensor([[1.0000, 0.9500],
        [0.9500, 1.0000]])
residualCovariance tensor([[1, 0],
        [0, 1]])
PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0300, 0.0200])
pdvthresh tensor([1.8808, 2.0537])
meanEffect tensor([0.4456, 0.2726])
meanEffectsAcrossAllGenes tensor([0.4456, 0.2726])
allEffects tensor([[-0.3997, -0.2246],
        [-0.4711, -0.2907],
        [-0.4944, -0.3293],
        ...,
        [-0.5948, -0.4194],
        [-0.4128, -0.2318],
        [-0.4164, -0.2337]])
PD1GivenV.mean() tensor(0.0307) PD2GivenV.mean() tensor(0.0205)
allEffects[i] tensor([-0.3997, -0.2246])
PDBothGivenV.mean tensor(0.0172, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.5606, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0307, 0.0205, 0.0172], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94930205]
 [0.9493020

In [7]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, DirichletMultinomial, Gamma, Beta, Categorical, Uniform, MultivariateNormal, Normal, LogNormal, Exponential, HalfCauchy, LKJCholesky
from jax import random
from numpyro.infer import MCMC, NUTS, SA, Predictive
import numpy as np
import jax.numpy as jnp
import jax

numpyro.set_host_device_count(6)
numpyro.enable_x64()

def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


# mu_exp, var_exp = get_log_params(liabParams55cov["afMean"].numpy(), 1)
# Expected number of components
# For 2 case types it's
# none, 1only, 2only, both
# For 3 it's
# none, 1only, 2only, 3only, 1&2, 1&3, 2&3, 123 (7)
# for 4 it's
# none, 1only, 2only, 3only, 4only, 1&2, 1&3, 1&4, 2&3, 2&4, 3&4, 123, 124, 134, 234, 1234
# which is 4 + 4choose2 + nchoose3  + nchoose4 
# nHypotheses = 6
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
empiricalAfs = altCounts.sum(1) / (nCases.sum() + nCtrls)
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])

# pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAllnp = [1 - samplePDs.sum(), *samplePDs]
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])

def model(data, nHypotheses: int, alpha = .05):
    # If in pyro: conc = numpyro.sample('conc', Exponential(exponential_prior).to_event(1))
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha / nHypotheses))

    with numpyro.plate("prob_plate", nHypotheses):
        conc = numpyro.sample('conc', Exponential(pdsAll).to_event(1))
        probs = numpyro.sample("probs", Dirichlet(conc))

    with numpyro.plate("data", data.shape[0]):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def model_mvn(data, nHypotheses: int, alpha = .05):
    # If in pyro: conc = numpyro.sample('conc', Exponential(exponential_prior).to_event(1))
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha / nHypotheses))

    with numpyro.plate("prob_plate", nHypotheses):
        # sigma = numpyro.sample("sigma", dist.Exponential(np.ones(kConditions)).to_event(1))
        # Rho = numpyro.sample("Rho", dist.LKJ(kConditions, kConditions))
        # cov = jnp.outer(sigma, sigma) * Rho
        # print("cov", cov)
        probs = jax.nn.softmax(numpyro.sample("probs", MultivariateNormal(0, np.eye(data.shape[1]))))

    with numpyro.plate("data", data.shape[0]):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

# def model(data):
#     # If in pyro: conc = numpyro.sample('conc', Exponential(exponential_prior).to_event(1))

#     with numpyro.plate("beta_plate", nHypotheses-1):
#         beta = numpyro.sample("beta", Beta(1, alpha))

#     with numpyro.plate("prob_plate", nHypotheses):
#         conc = numpyro.sample('conc', (np.ones(kConditions) / kConditions).to_event(1))
#         probs = numpyro.sample("probs", Dirichlet(conc)) * pdsAll

#     with numpyro.plate("data", data.shape[0]):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

# Does not work well
# [[0.42117703 0.3627187  0.10471608 0.11138819]
#  [0.7398306  0.14323903 0.08683579 0.03009459]
#  [0.97234545 0.00996042 0.01018163 0.00751249]
#  [0.35679422 0.35406996 0.20359637 0.08553945]
#  [0.47704746 0.13865462 0.29911017 0.08518775]
#  [0.60490041 0.17744334 0.17567386 0.0419824 ]]
# inferred stick-breaking weights [0.10809093 0.01804032 0.00217327 0.08453217 0.13801261 0.65321601]
# def modelPooled(data):
#     beta = numpyro.sample("beta", Beta(np.ones(nHypotheses-1), alpha, validate_args=False).to_event(1))

#     conc = numpyro.sample("conc", Gamma(pdsAllShaped, 1).to_event(2))
#     probs = numpyro.sample("probs", Dirichlet(conc*pdsAllShaped).to_event(1))
#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=probs[z], validate_args=False), obs=data)

def get_posterior_samples(mcmc):
    posterior_samples = mcmc.get_samples()
    posterior_samples['pi'] = stickbreak(posterior_samples['beta'])
    
    return posterior_samples

print("Ready")

Ready


In [8]:
mcmc_mvn = MCMC(NUTS(model_mvn, max_tree_depth=rho_mvn = posterior_samples_mvn["sigma"]
mcmc_mvn.run(random.PRNGKey(12269), altCounts, 12)
mcmc_mvn.print_summary()

sample: 100%|██████████| 1100/1100 [17:08<00:00,  1.07it/s, 127 steps of size 1.47e-02. acc. prob=0.92]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.67      0.03      0.67      0.62      0.73    249.67      1.01
    beta[1]      0.42      0.06      0.42      0.31      0.50    392.97      1.00
    beta[2]      0.67      0.10      0.67      0.52      0.84    233.68      1.01
    beta[3]      0.99      0.04      1.00      1.00      1.00    104.25      1.00
    beta[4]      1.00      0.00      1.00      1.00      1.00    218.11      1.01
    beta[5]      0.99      0.05      1.00      1.00      1.00     63.06      1.03
    beta[6]      0.97      0.10      1.00      0.94      1.00     69.89      1.00
    beta[7]      0.94      0.17      1.00      0.86      1.00     40.37      1.01
    beta[8]      0.93      0.17      1.00      0.72      1.00      9.28      1.22
    beta[9]      0.84      0.24      0.98      0.44      1.00     29.30    

In [10]:
# #https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
# from numpyro.infer import Predictive
# import arviz
posterior_samples_mvn = mcmc_mvn.get_samples()

# posterior_predictive = Predictive(model, posterior_samples_shallow)(random.PRNGKey(1), altCounts)
# prior = Predictive(model, num_samples=500)(random.PRNGKey(2), altCounts)

prob_mvn = posterior_samples_mvn["probs"]
print(jax.nn.softmax(prob_mvn).mean(0))

# rho_mvn = posterior_samples_mvn["sigma"]
# print(rho_mvn.mean(0))

beta_mvn = posterior_samples_mvn['beta']
weights_mvn = mix_weights(beta_mvn)
print("inferred stick-breaking weights mean: ", weights_mvn.mean(0))
print("inferred stick-breaking weights stdd: ", weights_mvn.std(0))



[[0.60593378 0.17635436 0.17363563 0.04407624]
 [0.40231207 0.38288013 0.1163476  0.09846019]
 [0.48432795 0.13204283 0.30420271 0.07942651]
 [0.31742947 0.34374417 0.25957352 0.07925284]
 [0.27104095 0.25652865 0.24081089 0.23161951]
 [0.22905035 0.24123777 0.2577164  0.27199548]
 [0.27207933 0.24701222 0.24590461 0.23500384]
 [0.23054149 0.26142115 0.2424045  0.26563286]
 [0.24026226 0.25954321 0.23821083 0.2619837 ]
 [0.25257603 0.2428562  0.25468136 0.24988641]
 [0.25550875 0.2454313  0.2517824  0.24727755]
 [0.24131773 0.26775832 0.23132063 0.25960331]]
inferred stick-breaking weights mean:  [6.69768694e-01 1.37684514e-01 1.30799332e-01 6.32198091e-02
 1.38195165e-03 9.96093144e-04 4.25789976e-04 9.77814841e-04
 3.91025459e-04 8.74617031e-04 5.07778317e-04 9.99000999e-04]
inferred stick-breaking weights stdd:  [0.03602987 0.02203326 0.03768891 0.01970046 0.03174924 0.03149844
 0.01346466 0.03092122 0.01236531 0.02765782 0.01605736 0.03159119]


In [142]:
import numpyro.distributions as dist
key = random.PRNGKey(0)
key, *split = random.split(key, 4)

# with numpyro.handlers.seed(rng_seed=4):
# a = numpyro.sample("a", dist.Normal(5, 2), rng_key=split[1])
# b = numpyro.sample("b", dist.Normal(-1, 0.5), rng_key=split[2])
sigma_cafe = numpyro.sample("sigma_cafe", dist.Exponential(1).expand([2]), rng_key=key)
Rho = numpyro.sample("Rho", dist.LKJ(2, 2), rng_key=split[0])
cov = jnp.outer(sigma_cafe, sigma_cafe) * Rho
# print(cov)

print(cov)



[[0.14112049 0.06191058]
 [0.06191058 1.24910286]]


In [31]:
init_rng_key = random.PRNGKey(12272)
mcmc1shallow = MCMC(NUTS(model, max_tree_depth=6), 100, 1000)
mcmc1shallow.run(init_rng_key, altCounts)
mcmc1shallow.print_summary()

sample: 100%|██████████| 1100/1100 [04:46<00:00,  3.84it/s, 63 steps of size 8.83e-02. acc. prob=0.93]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.66      0.03      0.67      0.62      0.72    655.99      1.00
   beta[1]      0.34      0.08      0.34      0.20      0.47    535.17      1.00
   beta[2]      0.40      0.09      0.40      0.25      0.54    705.66      1.00
   beta[3]      1.00      0.01      1.00      1.00      1.00    411.78      1.00
   beta[4]      0.93      0.16      1.00      0.74      1.00     77.02      1.02
 conc[0,0]      9.00      5.23      8.14      0.81     16.69    837.87      1.00
 conc[0,1]      3.29      2.07      2.86      0.22      6.33    776.05      1.00
 conc[0,2]      3.38      2.13      2.93      0.27      6.37    974.67      1.00
 conc[0,3]      1.35      0.92      1.14      0.11      2.66    714.13      1.00
 conc[1,0]      6.64      3.72      5.99      1.03     11.90    684.72      1.00
 conc

In [39]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow = mcmc1shallow.get_samples()

posterior_predictive = Predictive(model, posterior_samples_shallow)(random.PRNGKey(1), altCounts)
prior = Predictive(model, num_samples=500)(random.PRNGKey(2), altCounts)

prob1shallow = posterior_samples_shallow["probs"]
print(prob1shallow.mean(0))

betaScaledShallow = mcmc1shallow.get_samples()['beta']
weightsShallow = mix_weights(betaScaledShallow)
print("inferred stick-breaking weights mean: ", weightsShallow.mean(0))
print("inferred stick-breaking weights stdd: ", weightsShallow.std(0))



[[0.60656712 0.17642251 0.17525303 0.04175734]
 [0.42100628 0.36000387 0.10713235 0.1118575 ]
 [0.33178829 0.37489053 0.21369736 0.07962382]
 [0.47265644 0.137683   0.30226671 0.08739385]
 [0.23760629 0.27085968 0.27056135 0.22097268]
 [0.2548273  0.25773358 0.25803269 0.22940642]]
inferred stick-breaking weights mean:  [0.66354108 0.11508379 0.08999172 0.13439935 0.00110567 0.00100078]
inferred stick-breaking weights stdd:  [0.03528769 0.03073997 0.03062832 0.04267356 0.0315201  0.03159113]


NameError: name 'az' is not defined

In [52]:
# With 2x the number of components
# Speed appears to halve, but still > 2it/s, still takes only 63 steps per it
# actual result: 8:14s vs 04:46s for 6 components
nHypotheses = 12
init_rng_key = random.PRNGKey(12275)
mcmc1_shallow_12 = MCMC(NUTS(model, max_tree_depth=6), 100, 1000)
mcmc1_shallow_12.run(init_rng_key, altCounts)
mcmc1_shallow_12.print_summary()

sample: 100%|██████████| 1100/1100 [08:14<00:00,  2.23it/s, 63 steps of size 2.47e-02. acc. prob=0.94]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.14      0.04      0.14      0.07      0.21     28.23      1.05
    beta[1]      0.71      0.15      0.76      0.55      0.81      9.78      1.11
    beta[2]      0.59      0.15      0.60      0.38      0.84     34.84      1.01
    beta[3]      0.90      0.20      1.00      0.49      1.00      7.11      1.12
    beta[4]      0.93      0.21      1.00      0.79      1.00     15.93      1.07
    beta[5]      1.00      0.00      1.00      1.00      1.00    184.71      1.01
    beta[6]      0.85      0.27      1.00      0.36      1.00      7.72      1.23
    beta[7]      0.99      0.04      1.00      1.00      1.00     66.64      1.04
    beta[8]      0.94      0.15      1.00      0.78      1.00     58.61      1.07
    beta[9]      0.89      0.21      0.99      0.66      1.00     51.67     

In [55]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow_12 = mcmc1_shallow_12.get_samples()

posterior_predictive_shallow_12 = Predictive(model, posterior_samples_shallow_12)(random.PRNGKey(1), altCounts)
prior_shallow_12 = Predictive(model, num_samples=500)(random.PRNGKey(2), altCounts)

prob_shallow_12 = posterior_samples_shallow_12["probs"]
print(prob_shallow_12.mean(0))

beta_shallow_12 = posterior_samples_shallow_12['beta']
weights_shallow_12 = mix_weights(beta_shallow_12)
print("inferred stick-breaking weights mean: ", weights_shallow_12.mean(0))
print("inferred stick-breaking weights stdd: ", weights_shallow_12.std(0))

[[0.47690964 0.13687849 0.30033101 0.08588086]
 [0.60196606 0.17736425 0.17783865 0.04283104]
 [0.43859112 0.33704183 0.12103324 0.10333381]
 [0.32786191 0.37590634 0.21735709 0.07887466]
 [0.30105512 0.36907915 0.21658744 0.11327829]
 [0.0873675  0.28693257 0.42455969 0.20114025]
 [0.29310413 0.22862226 0.30566857 0.17260504]
 [0.19786615 0.20743047 0.28335852 0.31134487]
 [0.26997664 0.25169256 0.26654095 0.21178986]
 [0.2549449  0.33142057 0.22135155 0.19228298]
 [0.20179168 0.34715348 0.19236704 0.2586878 ]
 [0.17504335 0.34116427 0.27970352 0.20408886]]
inferred stick-breaking weights mean:  [0.14114367 0.60781908 0.15499018 0.08049369 0.01821113 0.00102538
 0.00080985 0.00095497 0.00093198 0.00090644 0.00094166 0.000999  ]
inferred stick-breaking weights stdd:  [0.0517402  0.12864444 0.1203839  0.02630912 0.04826102 0.03157819
 0.02560983 0.03019877 0.02947175 0.028664   0.02977787 0.03159119]


In [84]:
# With tree depth 7, roughly 2x as fast a 8?
nHypotheses = 12
init_rng_key = random.PRNGKey(12277)
mcmc1_shallow_12_7 = MCMC(NUTS(model, max_tree_depth=7), 100, 1000)
mcmc1_shallow_12_7.run(init_rng_key, altCounts)
mcmc1_shallow_12_7.print_summary()

sample: 100%|██████████| 1100/1100 [17:06<00:00,  1.07it/s, 127 steps of size 1.57e-02. acc. prob=0.94]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.09      0.03      0.09      0.05      0.13    153.58      1.00
    beta[1]      0.16      0.04      0.16      0.11      0.23    598.64      1.00
    beta[2]      0.15      0.03      0.15      0.10      0.19    174.50      1.00
    beta[3]      1.00      0.00      1.00      1.00      1.00    100.11      1.01
    beta[4]      0.95      0.15      1.00      0.84      1.00     43.40      1.09
    beta[5]      0.98      0.09      1.00      0.99      1.00     29.67      1.06
    beta[6]      0.99      0.08      1.00      1.00      1.00    114.38      1.00
    beta[7]      0.97      0.12      1.00      0.97      1.00    133.90      1.00
    beta[8]      0.99      0.06      1.00      1.00      1.00    265.70      1.00
    beta[9]      0.97      0.10      1.00      0.95      1.00    189.90    

In [85]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow_12_7 = mcmc1_shallow_12_7.get_samples()

posterior_predictive_shallow_12_7 = Predictive(model, posterior_samples_shallow_12_7)(random.PRNGKey(1), altCounts)
prior_shallow_12_7 = Predictive(model, num_samples=500)(random.PRNGKey(2), altCounts)

prob_shallow_12_7 = posterior_samples_shallow_12_7["probs"]
print(prob_shallow_12_7.mean(0))

beta_shallow_12_7 = posterior_samples_shallow_12_7['beta']
weights_shallow_12_7 = mix_weights(beta_shallow_12_7)
print("inferred stick-breaking weights mean: ", weights_shallow_12_7.mean(0))
print("inferred stick-breaking weights stdd: ", weights_shallow_12_7.std(0))

[[0.33173251 0.37678289 0.21114732 0.08033728]
 [0.47742279 0.14035764 0.29699542 0.08522415]
 [0.42176567 0.35977962 0.10612409 0.11233062]
 [0.60816102 0.17680399 0.17383013 0.04120486]
 [0.09221491 0.20257156 0.22548958 0.47972395]
 [0.07405939 0.19828925 0.1978399  0.52981147]
 [0.06524019 0.17453206 0.22806434 0.5321634 ]
 [0.09129762 0.15168894 0.21483506 0.54217838]
 [0.08602935 0.20386537 0.20439718 0.50570809]
 [0.05288289 0.23137158 0.1809478  0.53479773]
 [0.05803799 0.23189296 0.16591872 0.54415032]
 [0.09232233 0.20202711 0.18248785 0.5231627 ]]
inferred stick-breaking weights mean:  [0.09134486 0.14961421 0.11245278 0.64933292 0.00093485 0.00092969
 0.00095582 0.00099876 0.000999   0.00099735 0.000999   0.000999  ]
inferred stick-breaking weights stdd:  [0.03892508 0.04462906 0.03394091 0.03902477 0.02912067 0.02933935
 0.03022508 0.03158367 0.03159118 0.03153906 0.03159119 0.03159119]


In [80]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow_12_7 = mcmc1_shallow_12_7.get_samples()

posterior_predictive_shallow_12_7 = Predictive(model, posterior_samples_shallow_12_7)(random.PRNGKey(1), altCounts)
prior_shallow_12_7 = Predictive(model, num_samples=500)(random.PRNGKey(2), altCounts)

prob_shallow_12_7 = posterior_samples_shallow_12_7["probs"]
print(prob_shallow_12_7.mean(0))

beta_shallow_12_7 = posterior_samples_shallow_12_7['beta']
weights_shallow_12_7 = mix_weights(beta_shallow_12_7)
print("inferred stick-breaking weights mean: ", weights_shallow_12_7.mean(0))
print("inferred stick-breaking weights stdd: ", weights_shallow_12_7.std(0))

[[0.47812082 0.14007595 0.29641373 0.08538949]
 [0.33743373 0.37546813 0.20495509 0.08214305]
 [0.60787163 0.17700772 0.17392798 0.04119267]
 [0.42847248 0.35769411 0.09960923 0.11422418]
 [0.63533005 0.19351306 0.09153351 0.07962338]
 [0.52022331 0.2074242  0.2094564  0.06289608]
 [0.46145399 0.23988845 0.23891069 0.05974688]
 [0.55500881 0.14233603 0.27982428 0.02283089]
 [0.60969118 0.16947231 0.13347808 0.08735842]
 [0.30097555 0.51377649 0.07802635 0.10722161]
 [0.46955517 0.39332311 0.06483065 0.07229106]
 [0.6503142  0.20830758 0.07996386 0.06141436]]
inferred stick-breaking weights mean:  [0.1499326  0.1031971  0.64912474 0.10061613 0.00126433 0.00091556
 0.00099707 0.00095441 0.00099899 0.00099897 0.000999   0.000999  ]
inferred stick-breaking weights stdd:  [0.04574475 0.03706621 0.04469692 0.04074861 0.03123071 0.02765979
 0.03151594 0.03018101 0.03159096 0.0315902  0.03159118 0.03159119]


In [ ]:
# With .05 / nHypothesis prior
# With 2x the number of components and max_tree_depth 8 instead of 6.  1/4 as fast during warmup, but still may be acceptable
# DUring inference, it gets up to 1-2it/s, so up to 1/2 as fast. 127 steps, but sometimes 63

# Speed appears to halve, but still > 2it/s, still takes only 63 steps per it
# actual result: 12:30s vs 8:14s with tree depth 8
nHypotheses = 12
init_rng_key = random.PRNGKey(12276)
mcmc1_shallow_12 = MCMC(NUTS(model, max_tree_depth=7), 100, 1000)
mcmc1_shallow_12.run(init_rng_key, altCounts)
mcmc1_shallow_12.print_summary()

In [74]:
# With .05 / nHypothesis prior
# With 2x the number of components and max_tree_depth 8 instead of 6.  1/4 as fast during warmup, but still may be acceptable
# DUring inference, it gets up to 1-2it/s, so up to 1/2 as fast. 127 steps, but sometimes 63

# Speed appears to halve, but still > 2it/s, still takes only 63 steps per it
# actual result: 12:30s vs 8:14s with tree depth 8
nHypotheses = 12
alpha = .01/2
init_rng_key = random.PRNGKey(12276)
mcmc1_shallow_12 = MCMC(NUTS(model, max_tree_depth=8), 100, 1000)
mcmc1_shallow_12.run(init_rng_key, altCounts)
mcmc1_shallow_12.print_summary()

sample: 100%|██████████| 1100/1100 [29:51<00:00,  1.63s/it, 255 steps of size 6.20e-02. acc. prob=0.90]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.65      0.04      0.65      0.59      0.71    969.29      1.00
    beta[1]      0.42      0.06      0.42      0.31      0.51   1007.30      1.00
    beta[2]      0.58      0.13      0.59      0.36      0.79    860.24      1.00
    beta[3]      1.00      0.01      1.00      1.00      1.00    922.16      1.00
    beta[4]      0.97      0.11      1.00      0.96      1.00    983.82      1.00
    beta[5]      0.97      0.11      1.00      0.98      1.00   1053.54      1.00
    beta[6]      0.97      0.12      1.00      0.98      1.00    997.38      1.00
    beta[7]      0.97      0.13      1.00      0.97      1.00    916.78      1.00
    beta[8]      0.97      0.11      1.00      0.97      1.00    957.99      1.00
    beta[9]      0.98      0.09      1.00      0.98      1.00   1103.10    

In [75]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow_12_82 = mcmc1_shallow_12.get_samples()

posterior_predictive_shallow_12_82 = Predictive(model, posterior_samples_shallow_12_82)(random.PRNGKey(3), altCounts)
prior_shallow_12_82 = Predictive(model, num_samples=500)(random.PRNGKey(4), altCounts)

prob_shallow_12_82 = posterior_samples_shallow_12_82["probs"]
print(prob_shallow_12_82.mean(0))

beta_shallow_12_82 = posterior_samples_shallow_12_82['beta']
weights_shallow_12_82 = mix_weights(beta_shallow_12_82)
print("inferred stick-breaking weights mean: ", weights_shallow_12_82.mean(0))
print("inferred stick-breaking weights stdd: ", weights_shallow_12_82.std(0))


[[0.60779161 0.17695785 0.17398098 0.04126957]
 [0.47750379 0.13982881 0.29697568 0.08569172]
 [0.41972903 0.36059798 0.10820411 0.11146889]
 [0.32928953 0.37686015 0.21475458 0.07909574]
 [0.52268963 0.21860311 0.17717223 0.08153503]
 [0.5043688  0.19000033 0.22039062 0.08524025]
 [0.49696973 0.21380118 0.18990064 0.09932845]
 [0.53080489 0.19249831 0.19332191 0.08337489]
 [0.50705313 0.18690241 0.21933174 0.08671272]
 [0.58024046 0.14063027 0.18910173 0.09002753]
 [0.50351272 0.18639899 0.20779018 0.10229811]
 [0.51046051 0.19051043 0.19983452 0.09919455]]
inferred stick-breaking weights mean:  [0.65087375 0.14619406 0.11726908 0.08585446 0.00111522 0.00100087
 0.000999   0.000999   0.00099893 0.00089702 0.000999   0.000999  ]
inferred stick-breaking weights stdd:  [0.03883303 0.03012401 0.03366777 0.03950153 0.03159289 0.03159115
 0.03159119 0.03159119 0.03158901 0.02836618 0.03159117 0.03159119]


In [56]:
# With 2x the number of components and max_tree_depth 8 instead of 6.  1/4 as fast during warmup, but still may be acceptable
# DUring inference, it gets up to 1-2it/s, so up to 1/2 as fast. 127 steps, but sometimes 63

# Speed appears to halve, but still > 2it/s, still takes only 63 steps per it
# actual result: 12:30s vs 8:14s with tree depth 8
nHypotheses = 12
init_rng_key = random.PRNGKey(12276)
mcmc1_shallow_12 = MCMC(NUTS(model, max_tree_depth=8), 100, 1000)
mcmc1_shallow_12.run(init_rng_key, altCounts)
mcmc1_shallow_12.print_summary()

sample: 100%|██████████| 1100/1100 [12:30<00:00,  1.47it/s, 63 steps of size 1.21e-01. acc. prob=0.83]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.11      0.03      0.11      0.06      0.15   1011.14      1.00
    beta[1]      0.73      0.05      0.73      0.66      0.80   1571.55      1.00
    beta[2]      0.59      0.09      0.59      0.43      0.73    960.29      1.00
    beta[3]      1.00      0.01      1.00      1.00      1.00    508.80      1.00
    beta[4]      0.97      0.13      1.00      0.96      1.00    806.85      1.00
    beta[5]      0.97      0.11      1.00      0.98      1.00    894.43      1.00
    beta[6]      0.97      0.11      1.00      0.96      1.00    634.29      1.01
    beta[7]      0.98      0.11      1.00      0.98      1.00    821.44      1.00
    beta[8]      0.97      0.12      1.00      0.96      1.00   1112.06      1.00
    beta[9]      0.97      0.11      1.00      0.97      1.00    971.72     

In [60]:
#https://arviz-devs.github.io/arviz/getting_started/InferenceDataCookbook.html#cookbook
from numpyro.infer import Predictive
import arviz
posterior_samples_shallow_12_8 = mcmc1_shallow_12.get_samples()

posterior_predictive_shallow_12_8 = Predictive(model, posterior_samples_shallow_12_8)(random.PRNGKey(3), altCounts)
prior_shallow_12_8 = Predictive(model, num_samples=500)(random.PRNGKey(4), altCounts)

prob_shallow_12_8 = posterior_samples_shallow_12_8["probs"]
print(prob_shallow_12_8.mean(0))

beta_shallow_12_8 = posterior_samples_shallow_12_8['beta']
weights_shallow_12_8 = mix_weights(beta_shallow_12_8)
print("inferred stick-breaking weights mean: ", weights_shallow_12_8.mean(0))
print("inferred stick-breaking weights stdd: ", weights_shallow_12_8.std(0))


[[0.33837388 0.37515634 0.20368977 0.08278002]
 [0.60762657 0.17673843 0.17420694 0.04142806]
 [0.47559996 0.13940189 0.29901631 0.08598184]
 [0.42921905 0.35778335 0.09877124 0.11422636]
 [0.25127128 0.24542413 0.25702803 0.24627656]
 [0.24705352 0.25525046 0.25622663 0.24146939]
 [0.25370517 0.26227912 0.22330641 0.2607093 ]
 [0.25697685 0.24311631 0.25830912 0.24159772]
 [0.2391295  0.26437516 0.23770062 0.25879471]
 [0.2451957  0.26166862 0.242633   0.25050268]
 [0.23767351 0.26156451 0.2527936  0.24796837]
 [0.25492683 0.26556019 0.2260203  0.25349268]]
inferred stick-breaking weights mean:  [0.10602952 0.65449745 0.14368539 0.0987603  0.00116081 0.00100162
 0.000999   0.000999   0.000999   0.000999   0.00099877 0.000999  ]
inferred stick-breaking weights stdd:  [0.04005045 0.04798781 0.03658517 0.04039464 0.03160592 0.03159113
 0.03159107 0.03159119 0.03159119 0.03159119 0.03158394 0.03159119]


In [49]:
mcmc1shallow.get_samples()['probs'].shape

(1000, 6, 4)

In [62]:
numpyro_data_shallow = arviz.from_numpyro(
    mcmc1shallow,
    prior=prior,
    posterior_predictive=posterior_predictive,
    #coords={"school": np.arange(4)},
    #dims={"theta": ["school"]},
)
numpyro_data

ValueError: Incompatible shapes for broadcasting: ((12,), (6,))

In [37]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [23]:
init_rng_key = random.PRNGKey(12273)
mcmc1 = MCMC(NUTS(model), 100, 1000)
mcmc1.run(init_rng_key, altCounts)
mcmc1.print_summary()

sample: 100%|██████████| 1100/1100 [18:14<00:00,  1.01it/s, 255 steps of size 3.91e-02. acc. prob=0.85]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.09      0.03      0.09      0.05      0.14   1080.65      1.00
   beta[1]      0.12      0.03      0.12      0.08      0.17   1103.77      1.00
   beta[2]      0.19      0.04      0.18      0.12      0.26    842.57      1.00
   beta[3]      1.00      0.00      1.00      1.00      1.00    854.48      1.00
   beta[4]      0.96      0.14      1.00      0.96      1.00    843.71      1.00
 conc[0,0]      5.59      3.28      5.09      1.10     10.35   1099.94      1.00
 conc[0,1]      6.21      3.45      5.65      0.72     10.82    824.72      1.00
 conc[0,2]      3.82      2.39      3.35      0.37      7.01   1015.74      1.00
 conc[0,3]      2.10      1.34      1.86      0.08      3.89   1173.33      1.00
 conc[1,0]      6.72      3.91      6.09      0.91     12.24    919.07      1.01
 con

In [29]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
prob1 = mcmc1.get_samples()["probs"]
print(prob1.mean(0))

betaScaled1 = mcmc1.get_samples()['beta']
weights = mix_weights(betaScaled1)
print("inferred stick-breaking weights mean: ", weights.mean(0))
print("inferred stick-breaking weights stdd: ", weights.std(0))

# From mcmc1_shallow
# [[0.60656712 0.17642251 0.17525303 0.04175734]
#  [0.42100628 0.36000387 0.10713235 0.1118575 ]
#  [0.33178829 0.37489053 0.21369736 0.07962382]
#  [0.47265644 0.137683   0.30226671 0.08739385]
#  [0.23760629 0.27085968 0.27056135 0.22097268]
#  [0.2548273  0.25773358 0.25803269 0.22940642]]
# inferred stick-breaking weights mean:  [0.66354108 0.11508379 0.08999172 0.13439935 0.00110567 0.00100078]
# inferred stick-breaking weights stdd:  [0.03528769 0.03073997 0.03062832 0.04267356 0.0315201  0.03159113]


[[0.33270817 0.37618553 0.21040757 0.08069873]
 [0.4251387  0.35759294 0.10496525 0.11230311]
 [0.47797731 0.14030328 0.29655883 0.08516059]
 [0.60855083 0.1766796  0.1736899  0.04107967]
 [0.26369594 0.23635565 0.2559624  0.24398601]
 [0.26940583 0.24730769 0.2488597  0.23442678]]
inferred stick-breaking weights mean:  [0.0939588  0.11084216 0.15024856 0.64666142 0.0011599  0.0010021 ]
inferred stick-breaking weights stdd:  [0.04095458 0.03663405 0.04084136 0.03801109 0.03163265 0.03159112]


In [144]:
def plot_param_post(params, param_name, param_full_name, figsize=(12, 4),
                    truth=None, plot_trace=True):
    if plot_trace:
        plt.figure(figsize=figsize)
        
    param = params[param_name]

    if plot_trace:
        plt.subplot(1, 2, 1)
        
    plt.boxplot(param.T, whis=[2.5, 97.5], showmeans=True, showfliers=False)
    plt.xlabel('mixture components')
    plt.ylabel(param_full_name)
    plt.title('95% Credible Intervals for {}'.format(param_full_name))
    if truth is not None:
        for line in truth:
            plt.axhline(line, ls=':')

    if plot_trace:
        plt.subplot(1, 2, 2)
        plt.plot(param)
        plt.xlabel('iterations')
        plt.ylabel(param_full_name)
        plt.title('Trace plot of {}'.format(param_full_name))


In [147]:
n0 = len(liabData55cov["unaffectedGenes"])
n1 = len(liabData55cov["affectedGenes"][0])
n2 = len(liabData55cov["affectedGenes"][1])
nBoth = len(liabData55cov["affectedGenes"][2])

pisEmpirical = tensor([n0, n1, n2, nBoth]) / tensor(float((n0 + n1 + n2 + nBoth)))
pisEmpirical

tensor([0.7508, 0.1023, 0.0984, 0.0485])

In [20]:
liabData55cov["afs"][liabData55cov["unaffectedGenes"]].mean(0) / liabParams55cov["afMean"]

tensor([0.5946, 0.1784, 0.1784, 0.0476], dtype=torch.float64)

In [31]:
(liabData55cov["afs"][liabData55cov["unaffectedGenes"]] / empiricalAfs[liabData55cov["unaffectedGenes"]].T).sum(1)

ValueError: operands could not be broadcast together with shapes (15047,4) (20000,) 

In [28]:
(liabData55cov["afs"][liabData55cov["unaffectedGenes"]]).sum(1)

tensor([9.1548e-05, 1.1249e-04, 9.7812e-05,  ..., 1.1564e-04, 1.0539e-04,
        1.2242e-04], dtype=torch.float64)

In [30]:
empiricalAfs

array([8.33333333e-05, 1.07142857e-04, 9.52380952e-05, ...,
       1.42857143e-04, 9.52380952e-05, 1.19047619e-04])

In [11]:
pdv = (liabData55cov["afs"][liabData55cov["unaffectedGenes"]] / np.array([empiricalAfs[liabData55cov["unaffectedGenes"]]]).T).mean(0)
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.6348, 0.1904, 0.1904, 0.0508], dtype=torch.float64)
pdv scaled tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [12]:
pdv = (liabData55cov["afs"][liabData55cov["unaffectedGenes"]].mean(0) / np.array([empiricalAfs[liabData55cov["unaffectedGenes"]]]).T.mean(0))
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.6330, 0.1899, 0.1899, 0.0506], dtype=torch.float64)
pdv scaled tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [13]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]] / np.array([empiricalAfs[liabData55cov["affectedGenes"][0]]]).T).mean(0)
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv1 = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / liabParams55cov["afMean"]
pdv2 = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]] / np.array([empiricalAfs
[liabData55cov["affectedGenes"][0]]]).T).mean(0)
afAffected0Mean = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) 
print("pdv1", pdv1)# PD|V
print("pdv2", pdv2)# PD|V
print("afAffected0Mean", afAffected0Mean)
print("pdv1 scaled", pdv1/pdv1.sum())
print("pdv2 scaled", pdv2/pdv2.sum())

pdv tensor([0.4202, 0.3903, 0.1305, 0.1041], dtype=torch.float64)
pdv scaled tensor([0.4020, 0.3735, 0.1249, 0.0996], dtype=torch.float64)
pdv1 tensor([0.5729, 0.5453, 0.1782, 0.1454], dtype=torch.float64)
pdv2 tensor([0.4202, 0.3903, 0.1305, 0.1041], dtype=torch.float64)
afAffected0Mean tensor([5.7286e-05, 5.4534e-05, 1.7816e-05, 1.4542e-05], dtype=torch.float64)
pdv1 scaled tensor([0.3973, 0.3782, 0.1236, 0.1009], dtype=torch.float64)
pdv2 scaled tensor([0.4020, 0.3735, 0.1249, 0.0996], dtype=torch.float64)


In [14]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][1]] / np.array([empiricalAfs[liabData55cov["affectedGenes"][1]]]).T).mean(0)# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5050, 0.1542, 0.3121, 0.0832], dtype=torch.float64)
pdv scaled tensor([0.4788, 0.1462, 0.2960, 0.0789], dtype=torch.float64)


In [15]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5707, 0.5524, 0.3697, 0.1230], dtype=torch.float64)
pdv scaled tensor([0.3532, 0.3419, 0.2288, 0.0762], dtype=torch.float64)


In [16]:
pdvpv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) #/ liabParams55cov["afMean"]# PD|V
pdv = pdvpv / liabParams55cov["afMean"]
print("pdv", pdv)
print("pdvpv", pdvpv)
print("pdvpv.sum()", pdvpv.sum())
# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdvpv scaled", pdvpv/pdvpv.sum())


pdv tensor([0.5707, 0.5524, 0.3697, 0.1230], dtype=torch.float64)
pdvpv tensor([5.7068e-05, 5.5238e-05, 3.6966e-05, 1.2304e-05], dtype=torch.float64)
pdvpv.sum() tensor(0.0002, dtype=torch.float64)
pdvpv scaled tensor([0.3532, 0.3419, 0.2288, 0.0762], dtype=torch.float64)


In [17]:
liabData55cov["altCounts"][liabData55cov["affectedGenes"][2]]

tensor([[4., 4., 3., 0.],
        [4., 4., 5., 2.],
        [4., 6., 1., 1.],
        ...,
        [2., 6., 1., 1.],
        [5., 1., 4., 0.],
        [6., 5., 4., 1.]], dtype=torch.float64)

In [18]:
import matplotlib.pyplot as plot
%matplotlib inline

plt.plot(liabData55cov["altCounts"])

NameError: name 'plt' is not defined

In [14]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) 

tensor([5.7571e-05, 5.5241e-05, 3.7103e-05, 1.2332e-05], dtype=torch.float64)

In [15]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0) 

tensor([5.7334e-05, 5.5250e-05, 1.7842e-05, 1.4733e-05], dtype=torch.float64)

In [16]:
pdsAllnp

[0.5952380895614624, 0.17857143, 0.17857143, 0.04761905]

In [17]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [18]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([5.7334e-05, 5.5250e-05, 1.7842e-05, 1.4733e-05], dtype=torch.float64)

In [19]:
liabParams55cov["afMean"]

tensor(1.0000e-04)